# MobileNet

## Questions:
- **What are the advantages and tradeoffs of the MobileNet family of CNNs?**
    - MobileNet family of CNNs are models with significantly fewer parameters and smaller disk space when compared with other models such VGG16. MobileNet models requires smaller hardware requirements during training, using GPU is not optional. The tradeoff would be its accuracy. Since they have fewer parameters, the model may show results with  a reduced accuracy due to its limited weights. 

In [32]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
physical_devices = tf.config.experimental.list_physical_devices("GPU")
print(f"Num GPUs Available: {len(physical_devices)}")

Num GPUs Available: 1


In [34]:
mobile = tf.keras.applications.mobilenet.MobileNet()

In [35]:
def prepare_image(file):
    img_path = "/kaggle/input/sign-language-digits/Sign-Language-Digits-Dataset/"
    img = image.load(img_path + file, target_size=(224,224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

## Image Preparation
Dataset: https://github.com/ardamavi/Sign-Language-Digits-Dataset

- 10 classes (0-9)
- Class Data:
    - class 0: 205 images
    - class 1: 206 images
    - class 2: 206 images
    - class 3: 206 images
    - class 4: 207 images
    - class 5: 207 images
    - class 6: 207 images
    - class 7: 206 images
    - class 8: 208 images
    - class 9: 204 images

In [36]:
"""
The following code was run on a local jupyter notebook 
since it is difficult to files in Kaggle directories.
"""

# # Organize data into train, valid, test dirs

# os.chdir("data/Sign-Language-Digits-Dataset")
# if os.path.isdir("train/0/") is False:
#     os.mkdir("train")
#     os.mkdir("valid")
#     os.mkdir("test")

#     for i in range(0, 10):
#         shutil.move(f"{i}", "train")
#         os.mkdir(f"valid/{i}")
#         os.mkdir(f"test/{i}")

#         valid_samples = random.sample(os.listdir(f"train/{i}"), 30)
#         for filename in valid_samples:
#             shutil.move(f"train/{i}/{filename}", f"valid/{i}")
        
#         test_samples = random.sample(os.listdir(f"train/{i}"), 5)
#         for filename in test_samples:
#             shutil.move(f"train/{i}/{filename}", f"test/{i}")

# os.chdir("./../../")

'\nThe following code was run on a local jupyter notebook \nsince it is difficult to files in Kaggle directories.\n'

In [37]:
for i in range(0, 10):
    assert len(os.listdir(f"/kaggle/input/sign-language-digits-dataset/valid/{i}")) == 30
    assert len(os.listdir(f"/kaggle/input/sign-language-digits-dataset/test/{i}")) == 5

In [38]:
train_path = "/kaggle/input/sign-language-digits-dataset/train"
valid_path = "/kaggle/input/sign-language-digits-dataset/valid"
test_path = "/kaggle/input/sign-language-digits-dataset/test"

In [39]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    valid_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    test_path, target_size=(224,224), batch_size=10)

Found 1712 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
Found 50 images belonging to 10 classes.


In [40]:
assert train_batches.n == 1712
assert valid_batches.n == 300
assert test_batches.n == 50
assert train_batches.num_classes == valid_batches.num_classes == test_batches.num_classes == 10